In [163]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [164]:
df_train = pd.read_csv('input/application_train.csv',index_col='SK_ID_CURR')
df_train.info()
print('-----------')
df_test = pd.read_csv('input/application_test.csv',index_col='SK_ID_CURR')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Columns: 121 entries, TARGET to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 286.2+ MB
-----------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 48744 entries, 100001 to 456250
Columns: 120 entries, NAME_CONTRACT_TYPE to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(39), object(16)
memory usage: 45.0+ MB


# Bureau and Bureau Balance

## Bureau Balance

In [165]:
# load bureau balance file
df_balance = pd.read_csv('input/bureau_balance.csv')
df_balance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
SK_ID_BUREAU      int64
MONTHS_BALANCE    int64
STATUS            object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


In [166]:
display(df_balance[df_balance['SK_ID_BUREAU']==5715797].head(10))
display(df_balance['STATUS'].value_counts())

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
536,5715797,0,0
537,5715797,-1,1
538,5715797,-2,1
539,5715797,-3,0
540,5715797,-4,0
541,5715797,-5,0
542,5715797,-6,0
543,5715797,-7,2
544,5715797,-8,1
545,5715797,-9,0


C    13646993
0     7499507
X     5810482
1      242347
5       62406
2       23419
3        8924
4        5847
Name: STATUS, dtype: int64

In [ ]:
# select numeric statuses only
rows = (df_balance.STATUS!='X') & (df_balance.STATUS!='C')
cols = ['SK_ID_BUREAU','STATUS']
status = df_balance.loc[rows,cols].astype(int)

# aggregate status info
funcs = ['mean','max','first','std','sum'] # first gives most recent status
status = status.groupby('SK_ID_BUREAU').agg(funcs)

# rename columns
status.columns = ['BALANCE_'+'_'.join(x) for x in status.columns.values]

# reset index so SK_ID_BUREAU a separate column
status.reset_index(inplace=True)

status.head(10)

,SK_ID_BUREAU,BALANCE_STATUS_mean,BALANCE_STATUS_max,BALANCE_STATUS_first,BALANCE_STATUS_std,BALANCE_STATUS_sum
0,5001710,0.000000,0,0,0.000000,0
1,5001711,0.000000,0,0,0.000000,0
2,5001712,0.000000,0,0,0.000000,0
3,5001716,0.000000,0,0,0.000000,0
4,5001717,0.000000,0,0,0.000000,0
5,5001718,0.076923,1,0,0.271746,2
6,5001719,0.000000,0,0,0.000000,0
7,5001720,0.194444,1,1,0.401386,7
8,5001721,0.000000,0,0,0.000000,0
9,5001722,0.425532,1,0,0.499769,20


## Bureau

In [ ]:
# load bureau file
df_bureau = pd.read_csv('input/bureau.csv')
df_bureau.info()
display(df_bureau.head())
display(df_bureau.describe())

In [ ]:
# customer with apparently ~100 million credit, including 3 million for a car?
display(df_bureau[df_bureau.SK_ID_CURR==386819])

df_bureau.AMT_CREDIT_SUM.plot.box()

In [ ]:
print('bureau.csv unique ids:',df_bureau.SK_ID_BUREAU.nunique())
print('bureau_balance.csv unique ids:',df_balance.SK_ID_BUREAU.nunique())
print('bureau ids in bureau_balance:',df_bureau.SK_ID_BUREAU.isin(df_balance.SK_ID_BUREAU.unique()).sum())

In [ ]:
# merge status history from bureau_balance.csv with bureau.csv
df_bureau = df_bureau.merge(status, how='left', on='SK_ID_BUREAU')

display(df_bureau[status.columns].describe())

# don't need the bureau id anymore
df_bureau.drop('SK_ID_BUREAU',axis=1,inplace=True)

# don't need balance df anymore
del df_balance, status

In [ ]:
# shorten some strings
df_bureau['CREDIT_ACTIVE'] = df_bureau['CREDIT_ACTIVE'].replace({'Bad debt':'Bad'})
df_bureau['CREDIT_CURRENCY'] = df_bureau['CREDIT_CURRENCY'].str.replace('currency ','')
df_bureau['CREDIT_TYPE'] = df_bureau['CREDIT_TYPE'].replace({'Another type of loan':'Other',
                                                             'Car loan':'Car',
                                                             'Cash loan (non-earmarked)':'Cash',
                                                             'Consumer credit':'Consumer',
                                                             'Credit card':'Card',
                                                             'Interbank credit':'Interbank',
                                                             'Loan for business development':'Business',
                                                             'Loan for purchase of shares (margin lending)':'Shares',
                                                             'Loan for the purchase of equipment':'Equipment',
                                                             'Loan for working capital replenishment':'Capital',
                                                             'Mobile operator loan':'Mobile',
                                                             'Real estate loan':'Estate',
                                                             'Unknown type of loan':np.nan})

In [ ]:
# convert all categoricals to dummies
numeric_cols = [col for col in df_bureau.columns if df_bureau[col].dtype is not 'object']
df_bureau = pd.get_dummies(df_bureau,columns=['CREDIT_CURRENCY','CREDIT_TYPE'])

exclude_cols = ['SK_ID_CURR','CREDIT_ACTIVE']
dummy_cols = [col for col in df_bureau.columns if col not in numeric_cols and not in exclude_cols]
numeric_cols = [col for col in df_bureau.columns if col not in dummy_cols and not in exclude_cols]

df_bureau.describe()

In [ ]:
agg_funcs = {}
for col in dummy_cols:
    agg_funcs[col] = ['count','mean']

for col in numeric_cols:
    agg_funcs[col] = ['mean','max','min','std']

agg_funcs

In [ ]:
df_bureau.groupby(['SK_ID_CURR','CREDIT_ACTIVE']).agg(agg_funcs).info()

## OLD

In [ ]:
tmp=df_bureau.groupby('SK_ID_CURR').BALANCE_STATUS_mean.agg(['mean','max','min'])
(tmp['min']>0).value_counts()

In [ ]:
df_train['BUREAU_CNT_PREV_CREDIT'] = df_bureau.SK_ID_CURR.value_counts()
df_train['BUREAU_CNT_PREV_CREDIT'].fillna(0,inplace=True)

sns.kdeplot(df_train.loc[df_train['TARGET'] == 0, 'BUREAU_CNT_PREV_CREDIT'].dropna(), label='target == 0')
sns.kdeplot(df_train.loc[df_train['TARGET'] == 1, 'BUREAU_CNT_PREV_CREDIT'].dropna(), label='target == 1')
plt.xlim([-2.5,20])
plt.ylim([0,0.15])
plt.xlabel('BUREAU_CNT_PREV_CREDIT')
# people without previous credit more likely to default?

plt.figure(figsize=(15,5))
sns.barplot(x='BUREAU_CNT_PREV_CREDIT',y='TARGET',data=df_train)
plt.xticks(rotation=90);

In [ ]:
cnt_credit = df_bureau.groupby('SK_ID_CURR').CREDIT_ACTIVE.value_counts().unstack(fill_value=0)
cnt_credit.columns = 'BUREAU_CNT_CREDIT_'+cnt_credit.columns.values

df_train = df_train.merge(cnt_credit,how='left',left_index=True,right_index=True)

for col in cnt_credit.columns:
    plt.figure()
    sns.barplot(x=col,y='TARGET',data=df_train)
    plt.xticks(rotation=90)

In [ ]:
cnt_curr = df_bureau.groupby('SK_ID_CURR').CREDIT_CURRENCY.value_counts().unstack(fill_value=0)
cnt_curr.columns = 'BUREAU_CNT_'+cnt_curr.columns.values

df_train = df_train.merge(cnt_curr,how='left',left_index=True,right_index=True)

for col in cnt_curr.columns:
    plt.figure()
    sns.barplot(x=col,y='TARGET',data=df_train)
    plt.xticks(rotation=90)

In [ ]:
df_bureau['CREDIT_TYPE'].value_counts()

cnt_typ = df_bureau.groupby('SK_ID_CURR').CREDIT_TYPE.value_counts().unstack(fill_value=0)
cnt_typ.columns = 'BUREAU_CNT_TYPE_'+cnt_typ.columns.values

df_train = df_train.merge(cnt_typ,how='left',left_index=True,right_index=True)

for col in cnt_typ.columns:
    plt.figure()
    sns.barplot(x=col,y='TARGET',data=df_train)
    plt.xticks(rotation=90)

In [ ]:
df_train['BUREAU_DAY_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR').CREDIT_DAY_OVERDUE.max()
df_train['BUREAU_DAY_OVERDUE_MAX'].fillna(0,inplace=True)
sns.barplot(x='TARGET',y='BUREAU_DAY_OVERDUE_MAX',data=df_train)

df_train['BUREAU_DAY_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR').CREDIT_DAY_OVERDUE.mean()
df_train['BUREAU_DAY_OVERDUE_MEAN'].fillna(0,inplace=True)
plt.figure()
sns.barplot(x='TARGET',y='BUREAU_DAY_OVERDUE_MEAN',data=df_train)

df_train['BUREAU_DAY_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR').CREDIT_DAY_OVERDUE.min()
df_train['BUREAU_DAY_OVERDUE_MIN'].fillna(0,inplace=True)
plt.figure()
sns.barplot(x='TARGET',y='BUREAU_DAY_OVERDUE_MIN',data=df_train)

df_train['BUREAU_DAY_OVERDUE_STD'] = df_bureau.groupby('SK_ID_CURR').CREDIT_DAY_OVERDUE.std()
df_train['BUREAU_DAY_OVERDUE_STD'].fillna(0,inplace=True)
plt.figure()
sns.barplot(x='TARGET',y='BUREAU_DAY_OVERDUE_STD',data=df_train)

df_train['BUREAU_DAY_OVERDUE_SUM'] = df_bureau.groupby('SK_ID_CURR').CREDIT_DAY_OVERDUE.sum()
df_train['BUREAU_DAY_OVERDUE_SUM'].fillna(0,inplace=True)
plt.figure()
sns.barplot(x='TARGET',y='BUREAU_DAY_OVERDUE_SUM',data=df_train)


In [ ]:
df_train['BUREAU_DAYS_ENDDATE_MAX'] = df_bureau.groupby('SK_ID_CURR').DAYS_CREDIT_ENDDATE.max()
df_train['BUREAU_DAYS_ENDDATE_MAX'].fillna(0,inplace=True)
plt.figure()
sns.barplot(x='TARGET',y='BUREAU_DAYS_ENDDATE_MAX',data=df_train)

df_train['BUREAU_DAYS_ENDDATE_MEAN'] = df_bureau.groupby('SK_ID_CURR').DAYS_CREDIT_ENDDATE.mean()
df_train['BUREAU_DAYS_ENDDATE_MEAN'].fillna(0,inplace=True)
plt.figure()
sns.barplot(x='TARGET',y='BUREAU_DAYS_ENDDATE_MEAN',data=df_train)

df_train['BUREAU_DAYS_ENDDATE_MIN'] = df_bureau.groupby('SK_ID_CURR').DAYS_CREDIT_ENDDATE.min()
df_train['BUREAU_DAYS_ENDDATE_MIN'].fillna(0,inplace=True)
plt.figure()
sns.barplot(x='TARGET',y='BUREAU_DAYS_ENDDATE_MIN',data=df_train)